## 统计分析

通过指定统计分析字段，得到每个特征的p_value，所有的p_value计算都是基于Ttest计算。支持指定不同的分组`group`，例如train、val、test等分组统计。

对于两大类不同的特征

1. 离散特征，统计数量以及占比。
2. 连续特征，统计均值、方差。

In [ ]:
import pandas as pd
import numpy as np
from onekey_algo import OnekeyDS as okds
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.utils import print_join_info

def append20(x):
    if not x.startswith('X') and len(str(x)) == 6:
        return f"20{x}"
    return x

task = 'HRR_ANY'
# 修改成自己临床数据的文件。
test_data = pd.read_csv('data/CPGEA-TCGA 20230106 OK.csv')
isup = pd.read_csv('data/isup.csv')
test_data = pd.merge(test_data, isup, on='ID', how='inner')
stats_columns = ['age_at_diagnosis', 'psa_value', 'pathologic_T', 'pathologic_N', 'pathologic_M', 'residual_tumor', 'ISUP']
test_data = test_data[['ID', task, 'group'] + stats_columns]
test_data['ID'] = test_data['ID'].astype(str)
test_data['residual_tumor'] = test_data['residual_tumor'].astype(str)
test_data

In [ ]:
test_data.columns

# 数据映射

针对所有非数值形式的数据，可以进行类别映射。

In [ ]:
np.unique(test_data['residual_tumor'])

In [ ]:
pt_map = {'T2a': 2,
  'T2b': 2,
  'T2c': 2,
  'T3a': 3,
  'T3b': 3,
  'T4': 4,
  'pT2': 2,
  'pT2a': 2,
  'pT2b': 2,
  'pT2c': 2,
  'pT3a': 3,
  'pT3b': 3,
  'pT4': 4,
  'Tx':5}
test_data['pathologic_T'] = test_data['pathologic_T'].map(lambda x: pt_map[x] if x in pt_map else np.nan)

In [ ]:
pn_map = {'N0': 0,
  'N1': 1,
  'pN0': 0,
  'pN1': 1,
  'pNX': 2,
  'pNx': 2,
  'pn0': 0}
test_data['pathologic_N'] = test_data['pathologic_N'].map(lambda x: pn_map[x] if x in pn_map else np.nan)

In [ ]:
pm_map = {'pM0': 0, 'pM1a': 1, 'pM1b': 1, 'pMx': 2}
test_data['pathologic_M'] = test_data['pathologic_M'].map(lambda x: pm_map[x] if x in pm_map else np.nan)

In [ ]:
pm_map = {'NO': 1, 'R0': 1, 'R1': 1, 'YES': 0, 'R2':0, 'RX': 2, 'nan':2}
test_data['residual_tumor'] = test_data['residual_tumor'].map(lambda x: pm_map[x] if x in pm_map else np.nan)

In [ ]:
test_data

# 缺失值填充

In [ ]:
from onekey_algo.custom.components.comp1 import fillna
test_data = fillna(test_data, fill_mod='50%')
test_data

### 输出格式
支持两种格式数据，分别对应`pretty`参数的`True`和`False`, 当为`True`时，输出的是表格模式，反之则为dict数据。

```python
def clinic_stats(data: DataFrame, stats_columns: Union[str, List[str]], label_column='label',
                 group_column: str = None, continuous_columns: Union[str, List[str]] = None,
                 pretty: bool = True) -> Union[dict, DataFrame]:
    """

    Args:
        data: 数据
        stats_columns: 需要统计的列名
        label_column: 二分类的标签列，默认`label`
        group_column: 分组统计依据，例如区分训练组、测试组、验证组。
        continuous_columns: 那些列是连续变量，连续变量统计均值方差。
        pretty: bool, 是否对结果进行格式美化。

    Returns:
        stats DataFrame or json

    """
```

In [ ]:
from onekey_algo.custom.components.stats import clinic_stats
stats = clinic_stats(test_data, 
                     stats_columns= stats_columns,
                     label_column=task, 
                     group_column='group', 
                     continuous_columns= ['age_at_diagnosis', 'psa_value'], 
                     pretty=True)
stats.to_csv('stats.csv', index=False, encoding='utf_8_sig')
stats

In [ ]:
test_data[['ID', task, 'group'] + stats_columns].to_csv('features/clinical_sel.csv', index=False)

In [ ]:
test_data